参考：
* https://pytorch.apachecn.org/docs/1.4/blitz/autograd_tutorial.html
* https://mp.weixin.qq.com/s/kDGHQQ_wRQXds34HHCQNOQ  二、自动求梯度（非常重要）

PyTorch中，所有神经网络的核心是 autograd 包。先简单介绍一下这个包，然后训练我们的第一个的神经网络。

autograd 包为张量上的所有操作提供了自动求导机制。它是一个在运行时定义(define-by-run）的框架，这意味着反向传播是根据代码如何运行来决定的，并且每次迭代可以是不同的.

让我们用一些简单的例子来看看吧。

### 目录
1. 张量
2. 梯度




如果想了解数值微分数值积分和自动求导的知识，可以查看邱锡鹏老师的《神经网络与深度学习》第四章第五节：
下载地址：https://nndl.github.io/

在这里简单说说自动微分的原理吧：我们的目标是求
![](https://mmbiz.qpic.cn/mmbiz_svg/3a3QxMHZ8YwnLDxI5vXUicUGkLPGTzYj2ibhuKx1UP13WxmUdJgictKcxr6lOPtibsKl7ZGGeAwE24Ry6PrrCx3eo7IzUySWiaZFP/640?wx_fmt=svg&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

在  处的导数。我们的做法是利用链式法则分解为一系列的操作：

![](https://mmbiz.qpic.cn/mmbiz_png/vI9nYe94fsFfaUzHsEvJhbPArzshFk0NeicDibb2XiabzjkyicMPYJicicTEChPCX513zrDkiaZZFBiam6L7LtlJNffoEQ/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

![](https://mmbiz.qpic.cn/mmbiz_png/vI9nYe94fsFfaUzHsEvJhbPArzshFk0NZNia2AVN5haATojyPKKwH8atopTDJx2MHBd30maRWUW2HHyzz4F616w/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

![](https://mmbiz.qpic.cn/mmbiz_png/vI9nYe94fsFfaUzHsEvJhbPArzshFk0NIutMOaxtfibXItKfEVJiaSMM02K857BaEftQZwArA6f0NnoiaA41nLj8g/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)





## 1 张量及张量求导
torch.Tensor 是这个包的核心类。如果设置它的属性 .requires_grad 为 True，那么它将会追踪对于该张量的所有操作。当完成计算后可以通过调用 .backward()，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到.grad属性.

In [12]:
import torch

In [13]:
# 加入requires_grad=True参数可追踪函数求导
x = torch.ones(2,2,requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [14]:
# 对这个张量做一次运算：
y  = x + 2
print(y)
print(y.grad_fn) # # 创建了一个加法操作<AddBackward0 object at 0x0000017AF2F86EF0>


tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [15]:
# 像x这种直接创建的称为叶子节点，叶子节点对应的 grad_fn 是 None 。

print(x.is_leaf,y.is_leaf)

True False


In [16]:

# 整点复杂的操作
z = y * y * 3
out = z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [17]:
# .requires_grad_(...) 原地改变了现有张量的 requires_grad 标志。如果没有指定的话，默认输入的这个标志是 False。

a = torch.randn(2,2)    # 缺失情况下默认 requires_grad = False
a = ((a*3)/(a-1))
print(a.requires_grad)  # False
a.requires_grad_(True)  # 设置为True
print(a.requires_grad)
b = (a*a).sum()
print(b.grad_fn)

False
True


## 2 梯度



In [ ]:
# 现在开始进行反向传播，因为 out 是一个标量，因此 out.backward() 和 out.backward(torch.tensor(1.)) 等价。
out.backward()

In [19]:
print(x.grad) # 输出导数 d(out)/dx

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


我们的得到的是一个数取值全部为`4.5`的矩阵。

让我们来调用 `out` 张量 `o`。

就可以得到 $o = \frac{1}{4}\sum_i z_i$，$z_i = 3(x_i+2)^2$ 和 $z_i\bigr\rvert_{x_i=1} = 27$ 因此, $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$，因而 $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$。

数学上，若有向量值函数 $$\vec{y}=f(\vec{x})$$，那么 $\vec{y}$ 相对于 $\vec{x}$ 的梯度是一个雅可比矩阵：

$$
J=\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)
$$

通常来说，`torch.autograd` 是计算雅可比向量积的一个“引擎”。也就是说，给定任意向量 $v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$，计算乘积 $v^{T}\cdot J$。如果 $v$ 恰好是一个标量函数 $l=g\left(\vec{y}\right)$ 的导数，即 $v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$，那么根据链式法则，雅可比向量积应该是 $l$ 对 $\vec{x}$ 的导数：

$$
J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)
$$

(注意：行向量的$ v^{T}\cdot J$也可以被视作列向量的$J^{T}\cdot v$)

雅可比向量积的这一特性使得将外部梯度输入到具有非标量输出的模型中变得非常方便。

现在我们来看一个雅可比向量积的例子:

In [20]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)
# 在这种情况下，y 不再是标量。torch.autograd 不能直接计算完整的雅可比矩阵，但是如果我们只想要雅可比向量积，只需将这个向量作为参数传给 backward：

tensor([-282.4034, -374.6324, 1066.1123], grad_fn=<MulBackward0>)


In [21]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [22]:
# 也可以通过将代码块包装在 with torch.no_grad(): 中，来阻止autograd跟踪设置了 .requires_grad=True 的张量的历史记录。
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)


True
True
False
